# Converting Data Source Files to Database

---


I want to convert the source data (in CSV format) into a database to make it easier to analyze and manage the overall dataset.
I chose to use a DuckDB-style database as it works well with Pandas and is designed for analytical workflows. This will be particularly important when I am performing my EDA and feature engineering later in my workflow.

To support the use of a database, I will use each reservation's a universally unique identifier (UUID, created in a prior notebook) to use as the primary key for my tables. Then, I will split the source data into logical groups to replicate a live database, making it easier to query my data later.  

---

# Imports

In [1]:
import duckdb
import pandas as pd

# Load Data

In [2]:
data = pd.read_feather('../../data/source/reservations_with_uuids.feather')
data

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,...,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate,UUID
0,0,342,2015,July,27,1,0,0,2,0.0,...,NULL,NULL,0,Transient,0.00,0,0,Check-Out,2015-07-01,1502832c-f9fe-4c2b-a7be-cce3a85d5441
1,0,737,2015,July,27,1,0,0,2,0.0,...,NULL,NULL,0,Transient,0.00,0,0,Check-Out,2015-07-01,2f85c742-d1c8-44fb-84da-c6e3dbc039a2
2,0,7,2015,July,27,1,0,1,1,0.0,...,NULL,NULL,0,Transient,75.00,0,0,Check-Out,2015-07-02,cbe1e6ae-2b44-4de2-a634-72356a93617a
3,0,13,2015,July,27,1,0,1,1,0.0,...,304,NULL,0,Transient,75.00,0,0,Check-Out,2015-07-02,5a29a4bd-5718-425f-953e-0e18f3ff1d29
4,0,14,2015,July,27,1,0,2,2,0.0,...,240,NULL,0,Transient,98.00,0,1,Check-Out,2015-07-03,a020d6df-bcba-45ea-a1d7-02688d117fbc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79325,0,23,2017,August,35,30,2,5,2,0.0,...,394,NULL,0,Transient,96.14,0,0,Check-Out,2017-09-06,56bcbcf2-ef31-4986-bd3a-71a5f0257e37
79326,0,102,2017,August,35,31,2,5,3,0.0,...,9,NULL,0,Transient,225.43,0,2,Check-Out,2017-09-07,07fdd8a6-059a-4441-beb0-0f73f7e71faa
79327,0,34,2017,August,35,31,2,5,2,0.0,...,9,NULL,0,Transient,157.71,0,4,Check-Out,2017-09-07,c3d83a7c-8c2e-4233-a8e0-9b1f39bf989d
79328,0,109,2017,August,35,31,2,5,2,0.0,...,89,NULL,0,Transient,104.40,0,0,Check-Out,2017-09-07,bbd4055e-9d03-48e7-b4de-fc0a3b047a14


## Prepare Data Types for Database

In [8]:
data['ReservationStatusDate'] = pd.to_datetime(data['ReservationStatusDate'])
data['ReservationStatusDate']

0       2015-07-01
1       2015-07-01
2       2015-07-02
3       2015-07-02
4       2015-07-03
           ...    
79325   2017-09-06
79326   2017-09-07
79327   2017-09-07
79328   2017-09-07
79329   2017-09-07
Name: ReservationStatusDate, Length: 119390, dtype: datetime64[ns]

In [9]:
data.dtypes

IsCanceled                              int64
LeadTime                                int64
ArrivalDateYear                         int64
ArrivalDateMonth                       object
ArrivalDateWeekNumber                   int64
ArrivalDateDayOfMonth                   int64
StaysInWeekendNights                    int64
StaysInWeekNights                       int64
Adults                                  int64
Children                              float64
Babies                                  int64
Meal                                   object
Country                                object
MarketSegment                          object
DistributionChannel                    object
IsRepeatedGuest                         int64
PreviousCancellations                   int64
PreviousBookingsNotCanceled             int64
ReservedRoomType                       object
AssignedRoomType                       object
BookingChanges                          int64
DepositType                       

# Create Database and Add Data

## Define Custom Functions to Create DB Table

In [10]:
def map_dtype(dtype):
    """
    Map pandas dtype to DuckDB SQL type.

    Args:
        dtype (pandas.dtype): The dtype of the pandas DataFrame column.

    Returns:
        str: The corresponding DuckDB SQL type.
    """
    if pd.api.types.is_integer_dtype(dtype):
        return "INTEGER"
    elif pd.api.types.is_float_dtype(dtype):
        return "FLOAT"
    elif pd.api.types.is_bool_dtype(dtype):
        return "BOOLEAN"
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return "DATE"
    else:
        return "VARCHAR"


def generate_create_table_sql(df, table_name):
    """
    Generate a SQL CREATE TABLE statement based on the DataFrame schema.

    Args:
        df (pandas.DataFrame): The DataFrame to generate the table schema from.
        table_name (str): The name of the table to be created.

    Returns:
        str: The SQL CREATE TABLE statement.
    """
    columns = []
    
    for col, dtype in df.dtypes.items():
        duckdb_type = map_dtype(dtype)
        columns.append(f'"{col}" {duckdb_type}')
    create_table_sql = f'CREATE TABLE "{table_name}" ({", ".join(columns)});'
    
    return create_table_sql


def insert_dataframe_to_duckdb(con, df, table_name):
    """
    Insert a DataFrame into a DuckDB table.

    Args:
        con (duckdb.DuckDBPyConnection): The DuckDB connection object.
        df (pandas.DataFrame): The DataFrame to insert.
        table_name (str): The name of the table to insert the data into.
    """
    # Convert DataFrame to DuckDB compatible format and insert
    con.register("temp_df", df)
    con.execute(f"INSERT INTO \"{table_name}\" SELECT * FROM temp_df")
    con.unregister("temp_df")

## Create the Database

In [11]:
## Split the data into separate tables
tables = {
    "Reservations": data[['UUID', 'IsCanceled', 'LeadTime', 
                          'ArrivalDateYear', 'ArrivalDateMonth',
                          'ArrivalDateWeekNumber', 
                          'ArrivalDateDayOfMonth', 
                          'StaysInWeekendNights', 'StaysInWeekNights',
                          'BookingChanges', 'DaysInWaitingList', 'ADR',
                          'RequiredCarParkingSpaces', 
                          'TotalOfSpecialRequests','ReservationStatus',
                          'ReservationStatusDate', 'HotelName']],
    "Guests": data[['UUID', 'Adults', 'Children', 'Babies',
                    'IsRepeatedGuest', 'PreviousCancellations',
                    'PreviousBookingsNotCanceled', 'CustomerType']],
    "Rooms": data[['UUID', 'ReservedRoomType', 'AssignedRoomType']],
    "BookingAgents": data[['UUID', 'Agent', 'Company']],
    "BookingDetails": data[['UUID', 'Meal', 'Country', 'MarketSegment',
                            'DistributionChannel', 'DepositType']]
}

with duckdb.connect(database='../../data/reservations.duckdb') as con:
    for table_name, df in tables.items():
        create_table_sql = generate_create_table_sql(df, table_name)
        con.execute(create_table_sql)
        insert_dataframe_to_duckdb(con, df, table_name)

    # Verify the data insertion by querying a limited number of rows
    for table_name in tables.keys():
        result = con.execute(f'SELECT * FROM "{table_name}" LIMIT 5').fetchall()
        print(f"\nInserted DataFrame in DuckDB ({table_name}):")
        for row in result:
            print(row)



Inserted DataFrame in DuckDB (Reservations):
('1c6eb3a5-5485-4b3b-8c71-e144fad139e8', 0, 342, 2015, 'July', 27, 1, 0, 0, 3, 0, 0.0, 0, 0, 'Check-Out', datetime.date(2015, 7, 1), 'H1')
('3b57d230-1058-469a-abe4-dbed5b610989', 0, 737, 2015, 'July', 27, 1, 0, 0, 4, 0, 0.0, 0, 0, 'Check-Out', datetime.date(2015, 7, 1), 'H1')
('dfae4b2a-c728-432e-b040-7c319ea34f0a', 0, 7, 2015, 'July', 27, 1, 0, 1, 0, 0, 75.0, 0, 0, 'Check-Out', datetime.date(2015, 7, 2), 'H1')
('c1faed91-46de-480e-a9fb-3eb6b0a6587c', 0, 13, 2015, 'July', 27, 1, 0, 1, 0, 0, 75.0, 0, 0, 'Check-Out', datetime.date(2015, 7, 2), 'H1')
('af3d245d-d2ba-454c-860a-0bd073e372dd', 0, 14, 2015, 'July', 27, 1, 0, 2, 0, 0, 98.0, 0, 1, 'Check-Out', datetime.date(2015, 7, 3), 'H1')

Inserted DataFrame in DuckDB (Guests):
('1c6eb3a5-5485-4b3b-8c71-e144fad139e8', 2, 0.0, 0, 0, 0, 0, 'Transient')
('3b57d230-1058-469a-abe4-dbed5b610989', 2, 0.0, 0, 0, 0, 0, 'Transient')
('dfae4b2a-c728-432e-b040-7c319ea34f0a', 1, 0.0, 0, 0, 0, 0, 'Transient'

In [14]:
## Review and confirm results
with duckdb.connect(database='../../data/reservations.duckdb') as con:
    result = con.execute(f'SELECT * FROM Guests LIMIT 25').df()

result

,UUID,Adults,Children,Babies,IsRepeatedGuest,PreviousCancellations,PreviousBookingsNotCanceled,CustomerType
0,1c6eb3a5-5485-4b3b-8c71-e144fad139e8,2,0.0,0,0,0,0,Transient
1,3b57d230-1058-469a-abe4-dbed5b610989,2,0.0,0,0,0,0,Transient
2,dfae4b2a-c728-432e-b040-7c319ea34f0a,1,0.0,0,0,0,0,Transient
3,c1faed91-46de-480e-a9fb-3eb6b0a6587c,1,0.0,0,0,0,0,Transient
4,af3d245d-d2ba-454c-860a-0bd073e372dd,2,0.0,0,0,0,0,Transient
5,f5383af5-7330-41e6-9cff-61be6ae69989,2,0.0,0,0,0,0,Transient
6,3919d757-10e9-42e7-a2cb-cd15daca463b,2,0.0,0,0,0,0,Transient
7,3f85c0b5-2f5b-4751-8cd2-3ec7de56f9b3,2,0.0,0,0,0,0,Transient
8,c15d2615-2bef-41ed-b8fa-407aa798030b,2,0.0,0,0,0,0,Transient
9,e12527e7-35df-4c27-a480-a8653cd2f15f,2,0.0,0,0,0,0,Transient
